# UCI Wine Datasets

In [ ]:
import torch
from torch.utils.data import TensorDataset, DataLoader
import numpy as np

from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

print('torch:', torch.__version__)


In [ ]:
X, y = load_wine(return_X_y=True)
# If labels are categorical, keep as integers for classification
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)  # integer labels for classification
train_ds = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_ds, batch_size=32, shuffle=True)

Xb, yb = next(iter(train_loader))
print('batch X shape:', Xb.shape, 'batch y shape:', yb.shape)


## Train Without Regularization

In [ ]:
X_train_t = X_train.clone().detach()
y_train_t = y_train.clone().detach()

n_features = X_train_t.shape[1]
n_classes = int(y_train_t.max().item()) + 1

w = torch.zeros((n_features, n_classes), requires_grad=True)
b = torch.zeros(n_classes, requires_grad=True)

lr = 1e-2
epochs = 200
loss_fn = torch.nn.CrossEntropyLoss()

for _ in range(epochs):
    logits = X_train_t @ w + b
    loss = loss_fn(logits, y_train_t)
    loss.backward()
    with torch.no_grad():
        w -= lr * w.grad
        b -= lr * b.grad
        w.grad.zero_()
        b.grad.zero_()

print("Final training loss:", loss.item())

## Test Loss